<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/Deep-learning-examples-using-keras/blob/main/Natural_images_classification_from_scratch_acc%3E95_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !unzip -uq "/content/drive/My Drive/kaggle_dataset/natural_images.zip" -d "/content/drive/My Drive/kaggle_dataset/natural_images/"

In [ ]:
train_path = "/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images"

In [ ]:
import cv2 
im = cv2.imread("/content/drive/My Drive/kaggle_dataset/natural_images/resized/person/person_0001.jpg")
im.shape

In [ ]:
# Create directories and subdirectories to save resized images
import glob
import os
import shutil

for dir in glob.glob("/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images/*/*"):
  split= dir.split("/")
  path_folder = os.path.join("/content/drive/My Drive/kaggle_dataset/natural_images/resized/",split[8])
  if os.path.exists(path_folder):
    continue
  else:
    os.makedirs(path_folder)

In [ ]:
"""# Resize training images of 'n' classes
labels = ["car","motorbike","flower","fruit","person","cat","dog","airplane"]
resize_path_train = "/content/drive/My Drive/kaggle_dataset/natural_images/resized/"  
for idx, value in enumerate(labels):
    for im in glob.glob(os.path.join("/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images",value ,'*')):
      img  = cv2.imread(im)
      width = 300
      height = 300
      dim = (width, height)
      # resize image
      resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
      cv2.imwrite(os.path.join(resize_path_train, str(i), im.split("/")[-1]), resized)
      print(os.path.join(resize_path_train, value, im.split("/")[-1]))"""

In [ ]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images/',
                                                 target_size = (224,224),
                                                 batch_size =32,
                                                 class_mode = 'categorical',
                                                 subset="training")


test_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images/',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            subset="validation")

In [ ]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=128,padding="same",kernel_size=3, activation='relu', input_shape=[224,224, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a third convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=16,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(256, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=8, activation='softmax'))

In [ ]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.summary()

In [ ]:
history=cnn.fit_generator(
    training_set,
    epochs = 20,
    validation_data = test_set,
)

In [ ]:
model.save("/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images_cnn.h5")

In [ ]:
"""class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.85):
            print("\nReached >85% accuracy so cancelling training!")
            self.model.stop_training = True
        
callbacks = myCallback()"""

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# load and prepare the image
def load_image(filename):
  
	# load the image
	img = load_img(filename, target_size=(128,128))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 128,128, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 


In [ ]:
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/drive/My Drive/kaggle_dataset/natural_images/resized/airplane/airplane_0000.jpg')
	# load model
	model = load_model('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images_cnn.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
  
 
# entry point, run the example
run_example()